In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Initialize the AlexNet model
model = Sequential()

# Add the convolutional layers
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))
model.add(Conv2D(256, (5, 5), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))
model.add(Conv2D(384, (3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(384, (3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Flatten the output and add the fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       1

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define data augmentation for validation/testing
val_datagen = ImageDataGenerator(rescale=1./255)

# Load your dataset (replace 'train_dir' and 'val_dir' with your dataset directories)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Tomato_train_new',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 8061 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [ ]:
# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=40,
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator)
# )

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=25,
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator),
#     callbacks=[lr_scheduler, early_stopping]
# )

from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the best model
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/2_Alexnet_new.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model with the callback
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint]
)

Epoch 1/40
252/252 [==============================] - ETA: 0s - loss: 2.1668 - accuracy: 0.1750
Epoch 1: val_loss improved from inf to 2.05889, saving model to /content/drive/MyDrive/Colab Notebooks/2_Alexnet_new.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


252/252 [==============================] - 3315s 13s/step - loss: 2.1668 - accuracy: 0.1750 - val_loss: 2.0589 - val_accuracy: 0.2253
Epoch 2/40
252/252 [==============================] - ETA: 0s - loss: 1.8293 - accuracy: 0.3251
Epoch 2: val_loss did not improve from 2.05889
252/252 [==============================] - 127s 505ms/step - loss: 1.8293 - accuracy: 0.3251 - val_loss: 2.2116 - val_accuracy: 0.1852
Epoch 3/40
252/252 [==============================] - ETA: 0s - loss: 1.5463 - accuracy: 0.4400
Epoch 3: val_loss did not improve from 2.05889
252/252 [==============================] - 126s 502ms/step - loss: 1.5463 - accuracy: 0.4400 - val_loss: 2.2706 - val_accuracy: 0.2545
Epoch 4/40
252/252 [==============================] - ETA: 0s - loss: 1.2789 - accuracy: 0.5429
Epoch 4: val_loss improved from 2.05889 to 1.56335, saving model to /content/drive/MyDrive/Colab Notebooks/2_Alexnet_new.h5
252/252 [==============================] - 132s 525ms/step - loss: 1.2789 - accuracy: 0.54